In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
adnimerge=pd.read_csv('../data/ADNIMERGE_14Oct2024.csv')

BAN_ADNI=pd.read_csv('../results/BrainAgeNeXt/BAN_ADNI.csv')
DBN_ADNI=pd.read_csv('../results/DeepBrainNet/DBN_ADNI_BET.csv')
pyment_ADNI=pd.read_csv('../results/pyment/pyment_ADNI.csv')
ENIGMA_ADNI=pd.read_csv('../results/ENIGMA/ENIGMA_ADNI.csv')

C:\Users\parri\AppData\Local\Temp\ipykernel_32028\3576130418.py:1: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  adnimerge=pd.read_csv('../data/ADNIMERGE_14Oct2024.csv')


In [8]:
adnimerge=adnimerge[['PTID','PTRACCAT']]
BAN_ADNI

,ID,Predicted_Brain_Age,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded,Absolute_Error,Error
0,002_S_1155_I1270004,70.783458,MCI,M,70,y2,MRI,Accelerated Sagittal MPRAGE,Original,5/16/2019,DCM,11/08/2022,0.783458,0.783458
1,002_S_1155_I995496,71.316726,MCI,M,69,y1,MRI,Accelerated Sagittal MPRAGE,Original,5/08/2018,DCM,11/08/2022,2.316726,2.316726
2,002_S_1261_I989320,85.359909,CN,F,82,y1,MRI,Accelerated Sagittal MPRAGE,Original,4/24/2018,DCM,11/08/2022,3.359909,3.359909
3,002_S_1280_I829296,80.706638,CN,F,81,init,MRI,Accelerated Sagittal MPRAGE,Original,3/13/2017,DCM,11/08/2022,0.293362,-0.293362
4,002_S_4225_I1075136,75.972325,CN,M,77,init,MRI,Accelerated Sagittal MPRAGE,Original,11/12/2018,DCM,10/12/2022,1.027675,-1.027675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,941_S_6454_I1336238,75.498151,CN,F,75,y2,MRI,Accelerated Sagittal MPRAGE,Original,9/16/2020,DCM,10/13/2022,0.498151,0.498151
1206,941_S_6499_I1023542,76.182815,CN,F,69,sc,MRI,Accelerated Sagittal MPRAGE,Original,7/17/2018,DCM,10/12/2022,7.182815,7.182815
1207,941_S_6546_I1034852,71.810066,CN,F,80,sc,MRI,Accelerated Sagittal MPRAGE,Original,8/08/2018,DCM,10/12/2022,8.189934,-8.189934
1208,941_S_6580_I1046736,84.984787,CN,F,80,sc,MRI,Accelerated Sagittal MPRAGE,Original,9/07/2018,DCM,10/12/2022,4.984787,4.984787


In [13]:
BAN_ADNI['ID_prefix'] = BAN_ADNI['ID'].str[:10]
prefix_counts = BAN_ADNI['ID_prefix'].value_counts()
prefix_counts_gt1 = prefix_counts[prefix_counts > 1]
print(prefix_counts_gt1)

ID_prefix
014_S_2308    7
070_S_6236    7
116_S_4453    6
116_S_6550    6
022_S_5004    6
             ..
037_S_6187    2
037_S_6222    2
007_S_4272    2
041_S_1418    2
002_S_1261    2
Name: count, Length: 306, dtype: int64
